In [11]:
from analyze import *

# path = 'flight_data/5Aug_till_sys_id.csv'
# path = 'flight_data/aggressive_cmds2.csv'
# path = 'flight_data/Sep4_5inch_icra_5inch_20percent_randomized_3_3succesfull_runs.csv'
path = 'flight_data/Sep24_DummyDroneIdentification.csv'
path2 = 'flight_data/Sep24_DummyCircle.csv'

data = load_flight_data(path, new_format=True)
data2 = load_flight_data(path2, new_format=True)
# data = load_flight_data(path, new_format=False)
animate_data(data)

Loading flight_data/Sep24_DummyDroneIdentification.csv
Logging frequency: 496.2398236508749
Loading flight_data/Sep24_DummyCircle.csv
Logging frequency: 496.29922084275404


In [12]:
%matplotlib tk
plt.close('all')
# reload all functions from analyze.py
import sys
importlib.reload(sys.modules['analyze'])
from analyze import *

p1 = fit_thrust_drag_model(data2)
p2 = fit_moments_model(data)
p3 = fit_actuator_model(data)
# w_max = p3[1]
# print(p2[8]*w_max, p2[9]*w_max)

fitting thrust and drag model
fitting moments model
fitting actuator model...


In [13]:
%matplotlib
# plt.close('all')
imu_plot(data)
# ekf_plot(data)
# actuator_plot(data)

Using matplotlib backend: tkagg


In [14]:
# get absolute values of the parameters
p1 = np.abs(p1)
p2 = np.abs(p2)
p3 = np.abs(p3)

print('params = {')
print('\'k_w\': {:.2e}, \'k_x\': {:.2e}, \'k_y\': {:.2e},'.format(p1[0], p1[1], p1[2]))
print('\'k_p1\': {:.2e}, \'k_p2\': {:.2e}, \'k_p3\': {:.2e}, \'k_p4\': {:.2e},'.format(p2[0], p2[1], p2[2], p2[3]))
print('\'k_q1\': {:.2e}, \'k_q2\': {:.2e}, \'k_q3\': {:.2e}, \'k_q4\': {:.2e},'.format(p2[4], p2[5], p2[6], p2[7]))
params = [p2[8]]*4+[p2[9]]*4
print('\'k_r1\': {:.2e}, \'k_r2\': {:.2e}, \'k_r3\': {:.2e}, \'k_r4\': {:.2e}, \'k_r5\': {:.2e}, \'k_r6\': {:.2e}, \'k_r7\': {:.2e}, \'k_r8\': {:.2e},'.format(*params))
# print('\'k_r1\': {:.2e}, \'k_r2\': {:.2e},'.format(p2[8], p2[9]))
print('\'w_min\': {:.2f}, \'w_max\': {:.2f}, \'k\': {:.2f}, \'tau\': {:.2f}'.format(p3[0], p3[1], p3[2], p3[3]))
print('}')
w_max=p3[1]
print('params_normalized = {')
print('\'k_wn\': {:.2e}, \'k_xn\': {:.2e}, \'k_yn\': {:.2e},'.format(p1[0]*w_max**2, p1[1]*w_max, p1[2]*w_max))
print('\'k_p1n\': {:.2e}, \'k_p2n\': {:.2e}, \'k_p3n\': {:.2e}, \'k_p4n\': {:.2e},'.format(p2[0]*w_max**2, p2[1]*w_max**2, p2[2]*w_max**2, p2[3]*w_max**2))
print('\'k_q1n\': {:.2e}, \'k_q2n\': {:.2e}, \'k_q3n\': {:.2e}, \'k_q4n\': {:.2e},'.format(p2[4]*w_max**2, p2[5]*w_max**2, p2[6]*w_max**2, p2[7]*w_max**2))
print('\'k_rn\': {:.2e}, \'k_rdn\': {:.2e},'.format(p2[8]*w_max, p2[9]*w_max))
print('\'w_min\': {:.2f}, \'w_max\': {:.2f}, \'k\': {:.2f}, \'tau\': {:.2f}'.format(p3[0], p3[1], p3[2], p3[3]))
print('}')

params = {
'k_w': 1.94e-06, 'k_x': 5.67e-05, 'k_y': 6.59e-05,
'k_p1': 5.53e-05, 'k_p2': 5.22e-05, 'k_p3': 5.28e-05, 'k_p4': 5.32e-05,
'k_q1': 3.49e-05, 'k_q2': 4.63e-05, 'k_q3': 4.03e-05, 'k_q4': 3.81e-05,
'k_r1': 9.56e-03, 'k_r2': 9.56e-03, 'k_r3': 9.56e-03, 'k_r4': 9.56e-03, 'k_r5': 1.87e-03, 'k_r6': 1.87e-03, 'k_r7': 1.87e-03, 'k_r8': 1.87e-03,
'w_min': 258.92, 'w_max': 3342.82, 'k': 0.96, 'tau': 0.05
}
params_normalized = {
'k_wn': 2.17e+01, 'k_xn': 1.90e-01, 'k_yn': 2.20e-01,
'k_p1n': 6.18e+02, 'k_p2n': 5.83e+02, 'k_p3n': 5.90e+02, 'k_p4n': 5.95e+02,
'k_q1n': 3.90e+02, 'k_q2n': 5.17e+02, 'k_q3n': 4.50e+02, 'k_q4n': 4.26e+02,
'k_rn': 3.19e+01, 'k_rdn': 6.26e+00,
'w_min': 258.92, 'w_max': 3342.82, 'k': 0.96, 'tau': 0.05
}


In [ ]:
def randomization_big(num):
    # returns randomized parameters based on robin and till's values
    
    # actuator model
    w_min   = np.random.uniform(0,   500, size=num)
    w_max   = np.random.uniform(3000,5000, size=num)
    k       = np.random.uniform(0., 1.0, size=num)
    tau     = np.random.uniform(0.01, 0.1, size=num)
    
    # thrust and drag
    k_wn    = np.random.uniform(1.0e+01, 3.0e+01, size=num)
    k_xn    = np.random.uniform(1.0e-01, 3.0e-01, size=num)
    k_yn    = np.random.uniform(1.0e-01, 3.0e-01, size=num)
    
    # moments parameters nominal
    k_pn    = np.random.uniform(2.0e+02, 8.0e+02, size=num)
    k_qn    = np.random.uniform(2.0e+02, 8.0e+02, size=num)
    k_rn    = np.random.uniform(2.0e+01, 8.0e+01, size=num)
    k_rdn   = np.random.uniform(2.0e+00, 8.0e+00, size=num)
    
    # moments parameters variation per motor
    k_p1n   = k_pn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    k_p2n   = k_pn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    k_p3n   = k_pn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    k_p4n   = k_pn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    
    k_q1n   = k_qn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    k_q2n   = k_qn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    k_q3n   = k_qn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    k_q4n   = k_qn + np.random.uniform(-5.0e+01, 5.0e+01, size=num)
    
    k_r1n   = k_rn
    k_r2n   = k_rn
    k_r3n   = k_rn
    k_r4n   = k_rn
    k_r6n   = k_rdn
    k_r7n   = k_rdn
    k_r8n   = k_rdn
    
    # output parameters
    return [
        k_wn, k_xn, k_yn,
        k_p1n, k_p2n, k_p3n, k_p4n,
        k_q1n, k_q2n, k_q3n, k_q4n,
        k_r1n, k_r2n, k_r3n, k_r4n, k_r6n, k_r7n, k_r8n,
        w_min, w_max, k, tau
    ]

In [10]:
plt.close('all')

In [ ]:
# Robin:
params_5inch = {
'k_w': -2.49e-06, 'k_x': -4.85e-05, 'k_y': -7.28e-05,
'k_p1': -6.55e-05, 'k_p2': -6.61e-05, 'k_p3': 6.36e-05, 'k_p4': 6.67e-05,
'k_q1': -5.28e-05, 'k_q2': 5.86e-05, 'k_q3': -5.05e-05, 'k_q4': 5.89e-05,
'k_r1': 1.07e-02, 'k_r2': 1.07e-02, 'k_r3': 1.07e-02, 'k_r4': 1.07e-02, 'k_r5': 1.97e-03, 'k_r6': 1.97e-03, 'k_r7': 1.97e-03, 'k_r8': 1.97e-03,
'w_min': 238.49, 'w_max': 3295.50, 'k': 0.95, 'tau': 0.04
}

# Till:
params_3inch = {
'k_w': -6.00e-07, 'k_x': -3.36e-05, 'k_y': -3.73e-05,
'k_p1': -2.57e-05, 'k_p2': -2.51e-05, 'k_p3': 2.72e-05, 'k_p4': 2.00e-05,
'k_q1': -9.10e-06, 'k_q2': 9.96e-06, 'k_q3': -1.17e-05, 'k_q4': 8.21e-06,
'k_r1': 9.64e-03, 'k_r2': 9.64e-03, 'k_r3': 9.64e-03, 'k_r4': 9.64e-03, 'k_r5': 1.14e-03, 'k_r6': 1.14e-03, 'k_r7': 1.14e-03, 'k_r8': 1.14e-03,
'w_min': 305.40, 'w_max': 4887.57, 'k': 0.84, 'tau': 0.04
}


# print(params['k_w']*(params['w_max']**2)*0.4)

# Robin normalized:
params_normalized_5inch = {
'k_w': -2.71e+01, 'k_x': -1.60e-01, 'k_y': -2.40e-01,
'k_p1': -7.11e+02, 'k_p2': -7.18e+02, 'k_p3': 6.91e+02, 'k_p4': 7.24e+02,
'k_q1': -5.73e+02, 'k_q2': 6.37e+02, 'k_q3': -5.48e+02, 'k_q4': 6.40e+02,
'k_r': 3.52e+01, 'k_rd': 6.49e+00,
'w_min': 238.49, 'w_max': 3295.50, 'k': 0.95, 'tau': 0.04
}

# Till normalized:
params_normalized_3inch = {
'k_w': -1.43e+01, 'k_x': -1.64e-01, 'k_y': -1.82e-01,
'k_p1': -6.15e+02, 'k_p2': -5.98e+02, 'k_p3': 6.50e+02, 'k_p4': 4.79e+02,
'k_q1': -2.17e+02, 'k_q2': 2.38e+02, 'k_q3': -2.80e+02, 'k_q4': 1.96e+02,
'k_r': 4.71e+01, 'k_rd': 5.57e+00,
'w_min': 305.40, 'w_max': 4887.57, 'k': 0.84, 'tau': 0.04
}


# make a table like this
# | param      | 3 inch   | 5 inch   | param      | 3 inch   | 5 inch   |
# | k_w        | 1.23e-06 | 2.34e-06 | w_min      | 1.23e-06 | 2.34e-06 |
# | k_x        | 1.23e-06 | 2.34e-06 | w_max      | 1.23e-06 | 2.34e-06 |
# | k_y        | 1.23e-06 | 2.34e-06 | k          | 1.23e-06 | 2.34e-06 |
# |            |          |          | tau        | 1.23e-06 | 2.34e-06 |
# ---------------------------------------------------------------
# | k_p1       | 1.23e-06 | 2.34e-06 | k_r1       | 1.23e-06 | 2.34e-06 |
# | k_p2       | 1.23e-06 | 2.34e-06 | k_r2       | 1.23e-06 | 2.34e-06 |
# | k_p3       | 1.23e-06 | 2.34e-06 | k_r3       | 1.23e-06 | 2.34e-06 |
# | k_p4       | 1.23e-06 | 2.34e-06 | k_r4       | 1.23e-06 | 2.34e-06 |
# | k_q1       | 1.23e-06 | 2.34e-06 | k_r5       | 1.23e-06 | 2.34e-06 |
# | k_q2       | 1.23e-06 | 2.34e-06 | k_r6       | 1.23e-06 | 2.34e-06 |
# | k_q3       | 1.23e-06 | 2.34e-06 | k_r7       | 1.23e-06 | 2.34e-06 |
# | k_q4       | 1.23e-06 | 2.34e-06 | k_r8       | 1.23e-06 | 2.34e-06 |
# ---------------------------------------------------------------

# etc
# format to latex non-scientific notation
def float_to_latex(f):
    return str(np.round(f, 2))
text = f'''
\\begin{{table}}[H]
\\centering
\\begin{{tabular}}{{c|c|c|c|c|c}}
\\hline
param & 3 inch & 5 inch & param & 3 inch & 5 inch \\\\
\\hline
$\hat{{k}}_{{w}} $ & {float_to_latex(params_normalized_3inch['k_w'])} & {float_to_latex(params_normalized_5inch['k_w'])} & $\hat{{w}}_{{min}}$ & {float_to_latex(params_normalized_3inch['w_min'])} & {float_to_latex(params_normalized_5inch['w_min'])} \\\\
$\hat{{k}}_{{x}} $ & {float_to_latex(params_normalized_3inch['k_x'])} & {float_to_latex(params_normalized_5inch['k_x'])} & $\hat{{w}}_{{max}}$ & {float_to_latex(params_normalized_3inch['w_max'])} & {float_to_latex(params_normalized_5inch['w_max'])} \\\\
$\hat{{k}}_{{y}} $ & {float_to_latex(params_normalized_3inch['k_y'])} & {float_to_latex(params_normalized_5inch['k_y'])} & $\hat{{k}}$ & {float_to_latex(params_normalized_3inch['k'])} & {float_to_latex(params_normalized_5inch['k'])} \\\\
    & & & $\hat{{\\tau}}$ & {float_to_latex(params_normalized_3inch['tau'])} & {float_to_latex(params_normalized_5inch['tau'])} \\\\
\\hline
$\hat{{k}}_{{p1}}$ & {float_to_latex(params_normalized_3inch['k_p1'])} & {float_to_latex(params_normalized_5inch['k_p1'])} & $\hat{{k}}_{{r1}}$ & {float_to_latex(params_normalized_3inch['k_r'])} & {float_to_latex(params_normalized_5inch['k_r'])} \\\\
$\hat{{k}}_{{p2}}$ & {float_to_latex(params_normalized_3inch['k_p2'])} & {float_to_latex(params_normalized_5inch['k_p2'])} & $\hat{{k}}_{{r2}}$ & {float_to_latex(params_normalized_3inch['k_r'])} & {float_to_latex(params_normalized_5inch['k_r'])} \\\\
$\hat{{k}}_{{p3}}$ & {float_to_latex(params_normalized_3inch['k_p3'])} & {float_to_latex(params_normalized_5inch['k_p3'])} & $\hat{{k}}_{{r3}}$ & {float_to_latex(params_normalized_3inch['k_r'])} & {float_to_latex(params_normalized_5inch['k_r'])} \\\\
$\hat{{k}}_{{p4}}$ & {float_to_latex(params_normalized_3inch['k_p4'])} & {float_to_latex(params_normalized_5inch['k_p4'])} & $\hat{{k}}_{{r4}}$ & {float_to_latex(params_normalized_3inch['k_r'])} & {float_to_latex(params_normalized_5inch['k_r'])} \\\\
$\hat{{k}}_{{q1}}$ & {float_to_latex(params_normalized_3inch['k_q1'])} & {float_to_latex(params_normalized_5inch['k_q1'])} & $\hat{{k}}_{{r5}}$ & {float_to_latex(params_normalized_3inch['k_rd'])} & {float_to_latex(params_normalized_5inch['k_rd'])} \\\\
$\hat{{k}}_{{q2}}$ & {float_to_latex(params_normalized_3inch['k_q2'])} & {float_to_latex(params_normalized_5inch['k_q2'])} & $\hat{{k}}_{{r6}}$ & {float_to_latex(params_normalized_3inch['k_rd'])} & {float_to_latex(params_normalized_5inch['k_rd'])} \\\\
$\hat{{k}}_{{q3}}$ & {float_to_latex(params_normalized_3inch['k_q3'])} & {float_to_latex(params_normalized_5inch['k_q3'])} & $\hat{{k}}_{{r7}}$ & {float_to_latex(params_normalized_3inch['k_rd'])} & {float_to_latex(params_normalized_5inch['k_rd'])} \\\\
$\hat{{k}}_{{q4}}$ & {float_to_latex(params_normalized_3inch['k_q4'])} & {float_to_latex(params_normalized_5inch['k_q4'])} & $\hat{{k}}_{{r8}}$ & {float_to_latex(params_normalized_3inch['k_rd'])} & {float_to_latex(params_normalized_5inch['k_rd'])} \\\\
\\hline
\\end{{tabular}}
\\end{{table}}
'''

print(text)   
    

In [ ]:
#Tills simulation:
#mc.setInertia(m=0.4, I=np.diag([0.75e-3, 0.8e-3, 0.9e-3]))
#mc.addRotor(Rotor(r=[-0.05, +0.0635, 0.0], Tmax=5., kESC=0.5, tau=0.02, Izz=5e-7, dir='lh')) # RR
#mc.addRotor(Rotor(r=[+0.05, +0.0635, 0.0], Tmax=5., kESC=0.5, tau=0.02, Izz=5e-7, dir='rh')) # FR
#mc.addRotor(Rotor(r=[-0.05, -0.0635, 0.0], Tmax=5., kESC=0.5, tau=0.02, Izz=5e-7, dir='rh')) # RL
#mc.addRotor(Rotor(r=[+0.05, -0.0635, 0.0], Tmax=5., kESC=0.5, tau=0.02, Izz=5e-7, dir='lh')) # FL
#wmax=4900
params_mc = {
    'm':0.4,
    'Ixx':0.75e-3,
    'Iyy':0.8e-3,
    'Izz':0.9e-3,
    'r1': [-0.05, +0.0635, 0.0],
    'r2': [+0.05, +0.0635, 0.0],
    'r3': [-0.05, -0.0635, 0.0],
    'r4': [+0.05, -0.0635, 0.0],
    'Tmax':5,
    'cm': 0.01,
    'kESC':0.5,
    'tau':0.02,
    'Izz_rotor':5e-7,
    'w_max':4900
}

#Convert to our convention:
params = params_mc
#Tmax = k_w*w**2/m --> k_w = Tmax*m/w**2
params['k_w'] = (params['Tmax']/params['m'])/(params['w_max'])**2
params['k_x'] = 0
params['k_y'] = 0
params['k_p1'] = -params['k_w']*params['r1'][1]/params['Ixx']
params['k_p2'] = -params['k_w']*params['r2'][1]/params['Ixx']
params['k_p3'] = -params['k_w']*params['r3'][1]/params['Ixx']
params['k_p4'] = -params['k_w']*params['r4'][1]/params['Ixx']
params['k_q1'] =  params['k_w']*params['r1'][0]/params['Iyy']
params['k_q2'] =  params['k_w']*params['r2'][0]/params['Iyy']
params['k_q3'] =  params['k_w']*params['r3'][0]/params['Iyy']
params['k_q4'] =  params['k_w']*params['r4'][0]/params['Iyy']
params['k_r'] =  params['cm']*params['Tmax']/params['Izz']
params['k_rd'] =  params['Izz_rotor']/params['Izz']


for key in ['k_w', 'k_x', 'k_y', 'k_p1', 'k_p2', 'k_p3', 'k_p4', 'k_q1', 'k_q2', 'k_q3', 'k_q4', 'k_r', 'k_rd', 'w_min', 'w_max', 'k', 'tau']:
    if key in params.keys():
        if key == 'k_r':
            print('\'k_r1\': {:.2e}, \'k_r2\': {:.2e}, \'k_r3\': {:.2e}, \'k_r4\': {:.2e},'.format(params[key], params[key], params[key], params[key]))
        elif key == 'k_rd':
            print('\'k_r5\': {:.2e}, \'k_r6\': {:.2e}, \'k_r7\': {:.2e}, \'k_r8\': {:.2e},'.format(params[key], params[key], params[key], params[key]))
        else:
            print('\'{}\': {:.2e},'.format(key, params[key]))

# Normalize:
# params_n = {}
# params_n['k_w'] = params['k_w']*params['w_max']**2
# print(params_n)